In [5]:
import tensorflow as tf
import keras
import numpy as np

import os
import numpy as np

In [19]:
root_dir = r'E:\suvidha\retvec_bert\datasets\archive\DUTS-TR\DUTS-TR-Image'
image_names = [os.path.join(root_dir,i) for i in os.listdir(root_dir)]

def process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_png(image, 3)
    image_x = tf.image.resize(image, (64,64))
    image_y = tf.image.resize(image, (256,256))
    
    image_x = image_x * 2.0
    image_y = image_y * 2.0
    
    image_x = image_x / 255.0
    image_y = image_y / 255.0
    
    image_x = image_x - 1.0
    image_y = image_y - 1.0
    
    return image_x,image_y

def process_image_x(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_png(image, 3)
    image = tf.image.resize(image, (64,64))
    image = image * 2.0
    image = image / 255.0
    image = image - 1.0
    return image

def process_image_y(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_png(image, 3)
    image = tf.image.resize(image, (256,256))
    image = image * 2.0
    image = image / 255.0
    image = image - 1.0
    return image

def get_dataset(image_names,process_image,batch_size = 4):    
    dataset = tf.data.Dataset.from_tensor_slices(image_names)
    dataset = dataset.shuffle(batch_size * 10)
    dataset = dataset.map(process_image).batch(batch_size)
    return dataset


batch_size = 4
dataset = get_dataset(image_names,process_image,batch_size = batch_size)
# dataset_y = get_dataset(image_names,process_image_y,batch_size = batch_size)

In [13]:
class ResBlock(keras.layers.Layer):
    def __init__(self,filters,do_upsample=False,name=None,**kwargs):
        super().__init__(name=None,**kwargs)
        self.conv_1 = keras.layers.SeparableConv2D(filters,3,padding="same")
        self.conv_2 = keras.layers.Conv2D(filters,3,padding="same")

        self.conv_residual = keras.layers.SeparableConv2D(filters,3,padding="same")
        self.upsample = tf.keras.layers.UpSampling2D(2)
        self.do_upsample = do_upsample        

    def call(self,x):
        residual = x
        x = self.conv_1(x)
        x = self.conv_2(x)
        residual = self.conv_residual(residual)
        x = x + residual
        if self.do_upsample:
            x = self.upsample(x)

        x = keras.activations.silu(x)
        
        return x


input = keras.layers.Input((64,64,3),batch_size=None)

filters_orig = 64
filters = filters_orig
x = ResBlock(filters)(input)
x0 = ResBlock(filters,do_upsample=True)(x)

filters = filters_orig*2
x = ResBlock(filters,do_upsample=False)(x0)
x = ResBlock(filters,do_upsample=True)(x)

filters = filters_orig
x = ResBlock(filters,do_upsample=False)(x) + tf.keras.layers.UpSampling2D(2)(x0)
x = ResBlock(3,do_upsample=False)(x)

model = keras.Model(input,x,name="super_resolution")
# model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="mse")
epochs = 5
ckpt_path = "super_resolution.h5"
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    ckpt_path,
    save_weights_only=True,
    monitor="loss",
    mode="min",
)
model.fit(x=dataset, epochs=epochs, callbacks=[ckpt_callback])

Epoch 1/5
  10/3893 [..............................] - ETA: 7:54:46 - loss: 0.2350

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpsvngp60d\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpsvngp60d\assets
